In [115]:
import yfinance as yf
import alpha_vantage as av
import pandas as pd
import numpy as np
import regex as re
pd.set_option('display.max_rows', None)

In [116]:
PREV_YEARS_TO_CONSIDER_FCF = [0]    # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_OCF = [0]    # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_ROCE = [0]   # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_ROE = [0]    # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_D2E = [0]    # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_CR = [0] # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_PS = [0] # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_SG_1YR = [0] # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_SG_3YR = [0] # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_PG_1YR = [0] # put atleast [0] or maximum [0,-1,-2] years
PREV_YEARS_TO_CONSIDER_PG_3YR = [0] # put atleast [0] or maximum [0,-1,-2] years
INDUSTRY_PE = 20
INDEX_PE = 20
USE_INDEX_PE = False
PREV_QUARTERS_TO_CONSIDER_FOR_POSITIVE_SG = [0] # put atleast [0] or maximum [0,-1,-2] quarters
PREV_QUARTERS_TO_CONSIDER_FOR_POSITIVE_PG = [0] # put atleast [0] or maximum [0,-1,-2] quarters
PREV_QUARTERS_TO_CONSIDER_FOR_POSITIVE_OP_PG = [0] # put atleast [0] or maximum [0,-1,-2] quarters 

In [167]:
tickers=["HAL"]

for ticker in tickers:
    company = yf.Ticker(ticker)

In [118]:
list_of_info_keys = company.info.keys()
list_of_balance_sheet_keys = company.balance_sheet.index.tolist()
list_of_cashflow_keys = company.cashflow.index.tolist()
list_of_income_stmt_keys = company.income_stmt.index.tolist()
list_of_history_metadata_keys = company.history_metadata.keys()
list_of_financials = company.financials.T.keys()


def print_refined_list_of_keys(list_of_keys, expression):
    refind_list_of_keys = [x for x in list_of_keys if re.search(expression, x, re.IGNORECASE)]
    for key in refind_list_of_keys:
        print("-" , key)


def get_list_of_available_keys(expression):
    print("\nInfo Keys:\n")
    print_refined_list_of_keys(list_of_info_keys, expression)
    print("\nBalance Sheet Keys:\n")
    print_refined_list_of_keys(list_of_balance_sheet_keys, expression)
    print("\nCashflow Keys:\n")
    print_refined_list_of_keys(list_of_cashflow_keys, expression)
    print("\nIncome Statement Keys:\n")
    print_refined_list_of_keys(list_of_income_stmt_keys, expression)
    print("\nHistory Metadata Keys:\n")
    print_refined_list_of_keys(list_of_history_metadata_keys, expression)
    print("\nFinancials Keys:\n")
    print_refined_list_of_keys(list_of_financials, expression)

APPL: Period '1wk' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']


In [119]:
def calculate_free_cash_flow(company, year):
    free_cash_flow =company.cashflow.loc['Free Cash Flow']
    return free_cash_flow.iloc[0-year]

In [120]:
def is_free_cash_flow_positive(company):
    for year in PREV_YEARS_TO_CONSIDER_FCF:
        if calculate_free_cash_flow(company, year) < 0:
            return False
    return True

In [121]:
def calculate_operating_cash_flow(company, year):
    operating_cash_flow =company.cashflow.loc['Operating Cash Flow']
    return operating_cash_flow.iloc[0-year]

In [122]:
def is_operating_cash_flow_positive(company):
    for year in PREV_YEARS_TO_CONSIDER_OCF:
        if calculate_operating_cash_flow(company, year) < 0:
            return False
    return True

In [123]:
def calculate_roce(company, year):
    financials = company.financials
    balance_sheet = company.balance_sheet

    # Calculate EBIT (Earnings Before Interest and Taxes)
    ebit = financials.loc['EBIT']

    # Calculate Capital Employed (Total Assets - Current Liabilities)
    total_assets = balance_sheet.loc['Total Assets']
    current_liabilities = balance_sheet.loc['Current Liabilities']  # Using instead of Total Current Liabilities
    capital_employed = total_assets - current_liabilities

    # Calculate ROCE
    roce = ebit / capital_employed
    return roce.iloc[0-year]

In [124]:
# Check if ROCE is greater than or equal to 15%

def is_roce_greater_than_or_equal_to_15(company):
    for year in PREV_YEARS_TO_CONSIDER_ROCE:
        if calculate_roce(company, year) < 0.15:
            return False
    return True

In [125]:
def calculate_roe(company, year):
    net_income = company.income_stmt.loc['Net Income']
    shareholders_equity = company.balance_sheet.loc['Stockholders Equity'] # Using instead of Total Stockholders Equity
    roe = net_income / shareholders_equity
    return roe.iloc[0-year]

In [126]:
# Check if ROE is greater than or equal to 15%

def is_roe_greater_than_or_equal_to_15(company):
    for year in PREV_YEARS_TO_CONSIDER_ROE:
        if calculate_roe(company, year) < 0.15:
            return False
    return True

In [127]:
# Check if ROE is greater than or equal to 20%

def is_roe_greater_than_or_equal_to_20(company):
    for year in PREV_YEARS_TO_CONSIDER_ROE:
        if calculate_roe(company, year) < 0.2:
            return False
    return True

In [128]:
# Check if PE is less than industry PE

def calculate_trailing_pe(company):
    # Fetch the company's data
    info = company.info
    
    # Get the current market price
    current_price = info.get('currentPrice')
    
    # Get the trailing twelve months earnings per share
    eps_ttm = info.get('trailingEps')
    
    # Check if the necessary information is available
    if current_price is not None and eps_ttm is not None and eps_ttm != 0:
        # Calculate the trailing P/E ratio
        trailing_pe = current_price / eps_ttm
        return trailing_pe
    else:
        return "Required data to calculate trailing P/E is not available."
 

In [129]:
   
def is_pe_less_than_benchmark_pe(company, useindex = USE_INDEX_PE):
    # Calculate the trailing P/E ratio
    trailing_pe = calculate_trailing_pe(company)
        
    if useindex:
        industry_pe = INDEX_PE
    else:
        industry_pe = INDUSTRY_PE

    if trailing_pe < industry_pe:
        return True
    else:
        return False

In [130]:
def calculate_debt_to_equity_ratio(company, year):
    balance_sheet = company.balance_sheet
    total_liabilities = balance_sheet.loc['Current Liabilities'] # Using instead of Total Current Liabilities
    total_equity = balance_sheet.loc['Stockholders Equity'] # Using instead of Total Stockholders Equity
    debt_to_equity_ratio = total_liabilities / total_equity
    return debt_to_equity_ratio.iloc[0-year]


In [131]:
# Check if the company has a debt to equity ratio of less than 2

def is_debt_to_equity_ratio_less_than_2(company):
    for year in PREV_YEARS_TO_CONSIDER_D2E:
        if calculate_debt_to_equity_ratio(company, year) >= 2:
            return False
    return True 

In [132]:

# Check if the company has a debt to equity ratio of less than 0.3

def is_debt_to_equity_ratio_less_than_0_dot_3(company):
    for year in PREV_YEARS_TO_CONSIDER_D2E:
        if calculate_debt_to_equity_ratio(company, year) >= 0.3:
            return False
    return True 

In [133]:

# Check if the company has a debt to equity ratio of less than or equal to 0.2
def is_debt_to_equity_ratio_less_than_or_equal_to_0_dot_2(company):
    for year in PREV_YEARS_TO_CONSIDER_D2E:
        if calculate_debt_to_equity_ratio(company, year) > 0.2:
            return False
    return True 

In [134]:
def calculate_current_ratio(company, year):
    balance_sheet = company.balance_sheet
    total_current_assets = balance_sheet.loc['Current Assets'] # Using instead of Total Current Assets
    total_current_liabilities = balance_sheet.loc['Current Liabilities'] # Using instead of Total Current Liabilities
    current_ratio = total_current_assets / total_current_liabilities
    return current_ratio.iloc[0-year]


In [135]:
# Check if the company has a current ratio of greater than 2

def is_current_ratio_greater_than_2(company):
    for year in PREV_YEARS_TO_CONSIDER_CR:
        if calculate_current_ratio(company, year) <= 2:
            return False 
    return True 

In [136]:
def calculate_piotroski_f_score(company, year):
    financials = company.financials.T  # Transpose for easier row access
    balance_sheet = company.balance_sheet.T  # Transpose
    cash_flow = company.cash_flow.T  # Transpose

    n = 0-year

    # Initialize Piotroski score
    f_score = 0

    # Criteria 1: Positive net income
    f_score += 1 if financials['Net Income'].iloc[n] > 0 else 0

    # Criteria 2: Positive return on assets
    f_score += 1 if financials['Net Income'].iloc[n] / balance_sheet['Total Assets'].iloc[n] > 0 else 0

    # Criteria 3: Positive operating cash flow
    f_score += 1 if cash_flow['Operating Cash Flow'].iloc[n] > 0 else 0 # Using instead of Total Cash From Operating Activities

    # Criteria 4: Cash flow from operations greater than net income
    f_score += 1 if cash_flow['Operating Cash Flow'].iloc[n] > financials['Net Income'].iloc[n] else 0

    # Criteria 5: Lower ratio of long term debt in the current period compared to the previous one
    lt_debt_current = balance_sheet['Long Term Debt'].iloc[n]
    lt_debt_previous = balance_sheet['Long Term Debt'].iloc[n+1]
    f_score += 1 if lt_debt_current < lt_debt_previous else 0

    # Criteria 6: Higher current ratio this year compared to the previous year
    current_ratio_current = balance_sheet['Current Assets'].iloc[n] / balance_sheet['Current Liabilities'].iloc[n] # Using instead of Total Current Assets and Total Current Liabilities
    current_ratio_previous = balance_sheet['Current Assets'].iloc[n+1] / balance_sheet['Current Liabilities'].iloc[n+1] # Using instead of Total Current Assets and Total Current Liabilities
    f_score += 1 if current_ratio_current > current_ratio_previous else 0

    # Criteria 7: No new shares issued (compare the number of shares outstanding)
    # This criterion may require data not available in standard financial statements

    # Criteria 8: Higher gross margin compared to the previous year
    gross_margin_current = (financials['Total Revenue'].iloc[n] - financials['Cost Of Revenue'].iloc[n]) / financials['Total Revenue'].iloc[n]
    gross_margin_previous = (financials['Total Revenue'].iloc[n+1] - financials['Cost Of Revenue'].iloc[n+1]) / financials['Total Revenue'].iloc[n+1]
    f_score += 1 if gross_margin_current > gross_margin_previous else 0

    # Criteria 9: Higher asset turnover ratio compared to the previous year
    asset_turnover_current = financials['Total Revenue'].iloc[n] / balance_sheet['Total Assets'].iloc[n]
    asset_turnover_previous = financials['Total Revenue'].iloc[n+1] / balance_sheet['Total Assets'].iloc[n+1]
    f_score += 1 if asset_turnover_current > asset_turnover_previous else 0

    return f_score

In [137]:
# Check if piotroski f score is greater than 7

def is_piotroski_f_score_greater_than_7(company):
    for year in PREV_YEARS_TO_CONSIDER_PS:
        if calculate_piotroski_f_score(company, year) <= 7:
            return False
    return True

In [138]:
def calculate_sales_growth_for_1_yr(company, year):
    financials = company.financials.T  # Transpose for easier row access

    n = 0-year
    # Calculate one year sales growth
    try:
        current_year_sales = financials['Total Revenue'].iloc[n]
        previous_year_sales = financials['Total Revenue'].iloc[n+1]

        # Calculate sales growth in percentage
        sales_growth = ((current_year_sales - previous_year_sales) / previous_year_sales)
        return sales_growth
    except Exception as e:
        return f"Error calculating sales growth: {str(e)}"

In [139]:
# Check if slaes growth for 1 year is greater than or equal to 10%

def is_1yr_sales_growth_greater_than_or_equal_to_10(company):
    for year in PREV_YEARS_TO_CONSIDER_SG_1YR:
        if calculate_sales_growth_for_1_yr(company, year) < 0.1:
            return False
    return True

In [140]:
def calculate_sales_growth_for_3_yr(company, year):
    financials = company.financials.T  # Transpose for easier row access

    n = 0-year
    # Calculate one year sales growth
    try:
        current_year_sales = financials['Total Revenue'].iloc[n]
        third_preious_year_sales = financials['Total Revenue'].iloc[n+2]

        # Calculate sales growth in percentage
        sales_growth = ((current_year_sales - third_preious_year_sales) / third_preious_year_sales)
        return sales_growth
    except Exception as e:
        return f"Error calculating sales growth: {str(e)}"

In [141]:
# Check if slaes growth for 3 year is greater than or equal to 10%
    
def is_3yr_sales_growth_greater_than_or_equal_to_10(company):
    for year in PREV_YEARS_TO_CONSIDER_SG_3YR:
        if calculate_sales_growth_for_3_yr(company, year) < 0.1:
            return False
    return True

In [142]:
def calculate_profit_growth_for_1_yr(company, year):
    financials = company.financials.T  # Transpose for easier row access

    n = 0-year
    # Calculate one year profit growth
    try:
        current_year_profit = financials['Gross Profit'].iloc[n]
        previous_year_profit = financials['Gross Profit'].iloc[n+1]

        # Calculate profit growth in percentage
        profit_growth = ((current_year_profit - previous_year_profit) / previous_year_profit)
        return profit_growth
    except Exception as e:
        return f"Error calculating profit growth: {str(e)}"

In [143]:
# Check if profit growth for 1 year is greater than or equal to 10%
    
def is_1yr_profit_growth_greater_than_or_equal_to_10(company):
    for year in PREV_YEARS_TO_CONSIDER_PG_1YR:
        if calculate_profit_growth_for_1_yr(company, year) < 0.1:
            return False
    return True

In [144]:
def calculate_profit_growth_for_3_yr(company, year):
    financials = company.financials.T  # Transpose for easier row access

    n = 0-year
    # Calculate one year profit growth
    try:
        current_year_profit = financials['Gross Profit'].iloc[n]
        third_preious_year_profit = financials['Gross Profit'].iloc[n+2]

        # Calculate profit growth in percentage
        profit_growth = ((current_year_profit - third_preious_year_profit) / third_preious_year_profit)
        return profit_growth
    except Exception as e:
        return f"Error calculating profit growth: {str(e)}"

In [145]:
# Check if slaes growth for 3 year is greater than or equal to 10%
    
def is_3yr_profit_growth_greater_than_or_equal_to_10(company):
    for year in PREV_YEARS_TO_CONSIDER_PG_3YR:
        if calculate_profit_growth_for_3_yr(company, year) < 0.1:
            return False
    return True

In [146]:
def calculate_sales_growth_for_1_quarter(company, quarter):
    financials = company.quarterly_financials.T  # Transpose for easier row access

    n = 0-quarter
    # Calculate one quarter sales growth
    try:
        current_quarter_sales = financials['Total Revenue'].iloc[n]
        previous_quarter_sales = financials['Total Revenue'].iloc[n+1]

        # Calculate sales growth in percentage
        sales_growth = current_quarter_sales - previous_quarter_sales
        return sales_growth
    except Exception as e:
        return f"Error calculating sales growth: {str(e)}"

In [147]:
# Check if sales growth for previous quarters is positive 
    
def is_QOQ_sales_growth_positive(company):
    for quarter in PREV_QUARTERS_TO_CONSIDER_FOR_POSITIVE_SG:
        if calculate_sales_growth_for_1_quarter(company, quarter) < 0:
            return False
    return True

In [148]:
def calculate_profit_growth_for_1_quarter(company, year):
    financials = company.quarterly_financials.T  # Transpose for easier row access

    n = 0-year
    # Calculate one quarter profit growth
    try:
        current_quarter_profit = financials['Gross Profit'].iloc[n]
        previous_quarter_profit = financials['Gross Profit'].iloc[n+1]

        # Calculate profit growth in percentage
        profit_growth = current_quarter_profit - previous_quarter_profit
        return profit_growth
    except Exception as e:
        return f"Error calculating profit growth: {str(e)}"

In [149]:
# Check if profit growth for previous quarters is positive 
    
def is_QOQ_profit_growth_positive(company):
    for quarter in PREV_QUARTERS_TO_CONSIDER_FOR_POSITIVE_PG:
        if calculate_profit_growth_for_1_quarter(company, quarter) < 0:
            return False
    return True

In [150]:
def calculate_EBITDA_growth_for_1_quarter(company, year):
    financials = company.quarterly_financials.T  # Transpose for easier row access

    n = 0-year
    # Calculate one quarter EBITDA growth
    try:
        current_quarter_EBITDA = financials['EBITDA'].iloc[n]
        previous_quarter_EBITDA = financials['EBITDA'].iloc[n+1]

        # Calculate EBITDA growth in percentage
        EBITDA_growth = current_quarter_EBITDA - previous_quarter_EBITDA
        return EBITDA_growth
    except Exception as e:
        return f"Error calculating EBITDA growth: {str(e)}"

In [151]:
# Check if operating profit growth for previous quarters is positive 
    
def is_QOQ_operating_profit_growth_positive(company):
    for quarter in PREV_QUARTERS_TO_CONSIDER_FOR_POSITIVE_PG:
        if calculate_EBITDA_growth_for_1_quarter(company, quarter) < 0:
            return False
    return True

In [152]:
def do_prelimenary_checks_and_get_points(company):
    points = 0
    if is_roe_greater_than_or_equal_to_15(company):
        points += 1
    if is_roce_greater_than_or_equal_to_15(company):
        points += 1
    if is_debt_to_equity_ratio_less_than_or_equal_to_0_dot_2(company):
        points += 1
    if is_1yr_sales_growth_greater_than_or_equal_to_10(company):
        points += 1
    if is_3yr_sales_growth_greater_than_or_equal_to_10(company):
        points += 1
    if is_1yr_profit_growth_greater_than_or_equal_to_10(company):
        points += 1
    if is_3yr_profit_growth_greater_than_or_equal_to_10(company):
        points += 1
    if is_pe_less_than_benchmark_pe(company):
        points += 1
    if is_QOQ_sales_growth_positive(company):
        points += 1
    if is_QOQ_profit_growth_positive(company):
        points += 1
    if is_QOQ_operating_profit_growth_positive(company):
        points += 1

    return points

In [153]:
def do_management_parameters_check_and_get_points(company):
    points = 0

    if is_roce_greater_than_or_equal_to_15(company):
        points += 1
    if is_debt_to_equity_ratio_less_than_0_dot_3(company):
        points += 1
    if is_current_ratio_greater_than_2(company):
        points += 1
    if is_piotroski_f_score_greater_than_7(company):
        points += 1
    return points

In [154]:
def do_eps_check_and_get_points(company):
    points = 0

    FV = company.info.get('bookValue')
    eps = company.info.get('trailingEps')

    if eps >= 3*FV:
        points += 3
    elif eps >= 2*FV:
        points += 2
    elif eps >= FV:
        points += 1
    
    return points

In [155]:
def do_pe_multiples_nifty_check_and_get_points(company):
    points = 0

    if calculate_trailing_pe(company) > 20:
        points += 1
    elif calculate_trailing_pe(company) > 15:
        points += 2
    else:
        points += 3

    return points

In [156]:
def do_roe_check_and_get_points(company, year):
    points = 0

    if calculate_roe(company, year) > 22:
        points += 3
    elif calculate_roe(company, year) > 18:
        points += 2
    elif calculate_roe(company, year) > 15:
        points += 1
    return points
    

In [157]:
def do_dividend_yield_check_and_get_points(company):
    points = 0

    if company.info[ 'trailingAnnualDividendYield' ] > 0.015:
        points += 3
    elif company.info[ 'trailingAnnualDividendYield' ] > 0.010:
        points += 2
    elif company.info[ 'trailingAnnualDividendYield' ] > 0.005:
        points += 1
    return points

In [158]:
def do_roa_check_and_get_points(company):
    points = 0

    if company.info[ 'returnOnAssets' ] > 0.22:
        points += 3
    elif company.info[ 'returnOnAssets' ] > 0.16:
        points += 2
    elif company.info[ 'returnOnAssets' ] > 0.10:
        points += 1
    return points

In [159]:
def calculate_EVEBITDA(company):
    financials = company.financials.T  # Transpose for easier row access

    # Calculate EBITDA
    try:
        EBITDA = financials['EBITDA'].iloc[0]
        enterprise_value = company.info.get('enterpriseValue')
        EVEBITDA = enterprise_value / EBITDA
        return EVEBITDA
    except Exception as e:
        return f"Error calculating EVEBITDA: {str(e)}"

def do_evebitda_check_and_get_points(company):
    points = 0

    if calculate_EVEBITDA(company) < 9:
        points += 3
    elif calculate_EVEBITDA(company) < 12:
        points += 2
    elif calculate_EVEBITDA(company) < 15:
        points += 1

    return points

In [160]:
def do_index_parameter_scaling_and_get_points(company):
    points = 0

    points += do_management_parameters_check_and_get_points(company)
    points += do_eps_check_and_get_points(company)
    points += do_pe_multiples_nifty_check_and_get_points(company)
    points += do_roe_check_and_get_points(company, 0)
    points += do_dividend_yield_check_and_get_points(company)
    points += do_roa_check_and_get_points(company)
    points += do_evebitda_check_and_get_points(company)
    return points

In [168]:
try:
    print(do_prelimenary_checks_and_get_points(company))
except Exception as e:
    print("NULL")
print("Done")

9
Done
